![1](ATLASOD.gif)

## Pirma pamoka

Šioje pamokoje mes:
* Susipažinsime su dalelių fizikos CERN ROOT biblioteka
* Išmoksime įskaityti failą su ATLAS detektoriumi užfiksuotais susidūrimų duomenimis
* Atliksime imtis reikalingas studijuoti $Z\to\ell\ell$ procesą
* Apskaičiuosime leptonų sistemos keturmatį vektorių ir nupiešime masės spektros histogramą

Z bozonas skyla beveik akimirksniu, įmanoma užfiksuoti tik skilimo produktus

In [ ]:
# Įkraunama ROOT biblioteka
import ROOT
%jsroot on

In [ ]:
# Duomenų analizę pradedame pasitelkdami simuliacija. Atidarome ROOT failą
nuoroda = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/1largeRjet1lep/MC/mc_361106.Zee.1largeRjet1lep.root"
failas = ROOT.TFile.Open(nuoroda)
# Iš failo nuskaitome "medį" (tree) su susidūrimų duomenimis
medis = failas.Get("mini")

In [ ]:
# Atliekame paprastas operacijas su nuskaitytu medžiu: išvedame susidūrimų skaičių į ekraną
n_viso = medis.GetEntries()
print(f"Faile rasta {n_viso} įvykių")

In [ ]:
# Pažiūrėkime kokią informaciją galime rasti savo medyje
medis.Print()

### Kiekviena medžio "šaka" aprašo vis kitokį dalyką. Štai poros iš jų paaiškinimas:
`runNumber` - duomenų rinkimo periodo numeris  
`eventNumber` - unikalus įvykio identifikatorius  
`mcWeight` - simuliuoti įvykio svoriai reikalingi norint palyginti simuliuotus duomenis su tikrais duomenimis  
`lep_` - leptonų (elektronų ir muonų) duomenys. Pastebėkime, kad šie duomenys talpinami vektoriuje (`vector<float>`). Taip yra dėl to, kad kiekviename įvykyje gali susidaryti įvairus kiekis leptonų
`jet_` - hadroninių čiurkšlių duomenys  
`photon` - fotonų duomenys  

In [ ]:
# Visi įvykiai yra statistiškai nepriklausomi vienas nuo kito
# Pažiūrėkime, kaip atrodo pirmo įvykio leptonų skersinis impulsas (matuojami MeV)
# Naudojame for ciklą atlikti iteraciją, tačiau sustojame po pirmo įvykio 
for įvykis in medis:
    print(įvykis.lep_pt)
    break

In [ ]:
%timeit
# Kiekvienam įvykyje pabandome apskaičiuoti leptonų sistemos masę
# vienos-dimensijos, dvigubo tikslumo (H1D) histogramą leptonų sistemos masei
masės_histograma = ROOT.TH1D("masė", "masė", 100, 60, 120)
# Skaičiuojame, kiek įvykių pasirinkome (iš visų įmanomų)
n = 0
# Sukurkime po keturmatį vektorių abiems leptonų kandidatams
pirmas_leptonas = ROOT.TLorentzVector()
antras_leptonas = ROOT.TLorentzVector()

for įvykis in medis:
    # Įsitikiname, kad yra bent jau du leptonai
    # Kitaip praleidžiame šį įvykį
    if įvykis.lep_n < 2:
        continue
    # Leptonų krūviai privalo būti skirtingi
    if (įvykis.lep_charge[0] == įvykis.lep_charge[1]):
        continue   

    # Let's define one TLorentz vector for each, e.i. two vectors!
    pirmas_leptonas.SetPtEtaPhiE(įvykis.lep_pt[0]/1000., įvykis.lep_eta[0], įvykis.lep_phi[0], įvykis.lep_E[0]/1000.)
    antras_leptonas.SetPtEtaPhiE(įvykis.lep_pt[1]/1000., įvykis.lep_eta[1], įvykis.lep_phi[1], įvykis.lep_E[1]/1000.)
    # Next line: addition of two TLorentz vectors above --> ask mass very easy 
    # (devide by 1000 to get value in GeV)
    sistema = pirmas_leptonas + antras_leptonas

    masės_histograma.Fill(sistema.M())
    n += 1
    if n % 1000 == 0:
        print(f"Išanalizuota {n} įvykių")

print(f"Iš viso išanalizuota {n}/{n_viso} įvykių")

In [ ]:
drobė = ROOT.TCanvas()

In [ ]:
# Sekantis etapas sukurti drobę (canvas) ir ją nupiešti.
# Drobė savaime būna tuščia, taigi nupiešiame ir histogramą
drobė.Draw()
masės_histograma.GetXaxis().SetTitle("Leptonų sistemos masė [GeV]")
masės_histograma.SetLineColor(ROOT.kRed)
masės_histograma.Draw()

Sveikiname, aiškiai matome Z bozono rezonansą (90 GeV).  
__Užduotis__: ar galite nupiešti dvi histogramas, vieną atskirai $Z\to\mu^+\mu^-$ skilimui, o kitą $Z\to e^+e^-$?  
_Patarimas_: antrą histogramą virš pirmos galite nupiešti naudojantis "SAME" komanda, pvz: `histograma2.Draw("SAME")`